In [ ]:
import nimblephysics as nimble
gui = nimble.NimbleGUI()
gui.serve(8080)

In [ ]:
import nimblephysics as nimble
# rajagopal_opensim = nimble.biomechanics.OpenSimParser.parseOsim('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/skel_models_v1.1/bsm.osim')
skeleton = nimble.biomechanics.OpenSimParser.parseOsim('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/bioamass_v1.0/CMU/01/ab_fits/Models/optimized_scale_and_markers.osim',\
                                                                '/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/skel_models_v1.1/Geometry/').skeleton
skeleton_ref = nimble.biomechanics.OpenSimParser.parseOsim('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/bioamass_v1.0/CMU/01/ab_fits/Models/optimized_scale_and_markers.osim',\
                                                                '/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/skel_models_v1.1/Geometry/').skeleton
p0 = skeleton.getPositions()

In [ ]:
motion = nimble.biomechanics.OpenSimParser.loadMot(skeleton,'/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/bioamass_v1.0/CMU/02/ab_fits/IK/02_ik.mot')
gui.nativeAPI().renderSkeleton(skeleton,layer='1') 
[skeleton.getJoint(i).getDofName(j) for i in range(skeleton.getNumJoints()) for j in range(skeleton.getJoint(i).getNumDofs())]


In [ ]:
# Look for constraints
import numpy as np
p = p0.copy()

p[46] =np.pi/2

skeleton.setPositions(p)
gui.nativeAPI().renderSkeleton(skeleton) 

In [ ]:
# Qualitative analysis of filtering the pseudo ground truth
from scipy.signal import savgol_filter
import time
import numpy as np

motion_refined = motion.poses.copy()

# motion_refined[29:32,:] = np.tile(p0[29:32].reshape(3,1),motion_refined.shape[1]) # Scapula R
# motion_refined[39:42,:] = np.tile(p0[39:42].reshape(3,1),motion_refined.shape[1]) # Scapula L
# motion_refined[20:22,:] = np.tile(p0[20:22].reshape(2,1),motion_refined.shape[1]) # Thorax
# motion_refined[23:25,:] = np.tile(p0[23:25].reshape(2,1),motion_refined.shape[1]) # Thorax
# motion_refined[26:29,:] = np.tile(p0[26:29].reshape(3,1),motion_refined.shape[1]) # Head

nimble_joint_names = [ 'walker_knee_r', 'wrist_l', 'hip_r', 'GlenoHumeral_r', 'elbow_l', 'hip_l', 'elbow_r', 'wrist_r',  \
                        'walker_knee_l', 'GlenoHumeral_l', 'ankle_r', 'ankle_l']

for i in range(motion_refined.shape[1]):
    skeleton_ref.setPositions(motion.poses[:,i]) # target
    target = skeleton_ref.getJointWorldPositions([skeleton_ref.getJoint(l) for l in nimble_joint_names])
    q = skeleton.getPositions()
    step_loss = 0.05
    older_loss = np.inf
    for j in range(1000):
        pos = np.array(skeleton.getJointWorldPositions([skeleton.getJoint(l) for l in nimble_joint_names]))
        
        error = pos - target
        loss = np.inner(error, error)
        if np.abs(older_loss - loss) < 0.00001:
            break
        older_loss = loss
        
        d_loss_d__pos = 2 * (pos - target)
        d_pos_d_joint_angles = skeleton.getJointWorldPositionsJacobianWrtJointPositions([skeleton.getJoint(l) for l in nimble_joint_names])
        d_loss_d_joint_angles = d_pos_d_joint_angles.T @ d_loss_d__pos
        d_loss_d_joint_angles[29:32] = 0
        d_loss_d_joint_angles[39:42] = 0
        d_loss_d_joint_angles[20:22] = 0
        d_loss_d_joint_angles[23:25] = 0
        d_loss_d_joint_angles[26:29] = 0
        q -= step_loss * d_loss_d_joint_angles
        
        # Right Hip
        q[6] = max(min(q[6], 140*np.pi/180), -40*np.pi/180)     # [-40,140]
        q[7] = max(min(q[7], 45*np.pi/180), -45*np.pi/180)     # [-40,140]
        q[8] = max(min(q[8], 45*np.pi/180), -45*np.pi/180)     # [-40,140]

        # Left Hip
        q[13] = max(min(q[13], 140*np.pi/180), -40*np.pi/180)     # [-40,140]
        q[14] = max(min(q[14], 45*np.pi/180), -45*np.pi/180)     # [-40,140]
        q[15] = max(min(q[15], 45*np.pi/180), -45*np.pi/180)     # [-40,140]

        # Right Knee
        q[9] = max(min(q[9], 140*np.pi/180), -10*np.pi/180)     # [-10,140]

        # Left Knee
        q[16] = max(min(q[16], 140*np.pi/180), -10*np.pi/180)     # [-10,140]


        # Ankles
        q[10] = max(min(q[10], 55*np.pi/180), -20*np.pi/180)     # [-10,140]
        q[17] = max(min(q[17], 55*np.pi/180), -20*np.pi/180)     # [-10,140]

        # Lumbar amd thorax bending
        q[20] = max(min(q[20], 20*np.pi/180), -20*np.pi/180)     # [-10,140]
        q[23] = max(min(q[23], 20*np.pi/180), -20*np.pi/180)     # [-10,140]
        # Lumbar amd thorax extension
        q[21] = max(min(q[21], 15*np.pi/180), -50*np.pi/180)     # [-10,140]
        q[24] = max(min(q[24], 15*np.pi/180), -50*np.pi/180)     # [-10,140]
        # Lumbar amd thorax extension
        q[22] = max(min(q[22], 5*np.pi/180), -5*np.pi/180)     # [-10,140]
        q[25] = max(min(q[25], 5*np.pi/180), -5*np.pi/180)     # [-10,140]


        # Left Shoulder
        q[42] = max(min(q[42], 150*np.pi/180), 0*np.pi/180)    # [0,150]
        q[43] = max(min(q[43], 70*np.pi/180), -90*np.pi/180)    # [-70,90]
        q[44] = max(min(q[44], 180*np.pi/180), -60*np.pi/180)    #  [-60,180]
        
        # Right Shoulder
        q[32] = max(min(q[32], 0*np.pi/180), -150*np.pi/180)    # [0,150]
        q[33] = max(min(q[33], 90*np.pi/180), -70*np.pi/180)    # [-70,90]
        q[34] = max(min(q[34], 180*np.pi/180), -60*np.pi/180)    #  [-60,180]

        # elbow range of motion"
        q[45] = max(min(q[45], 154*np.pi/180), -6*np.pi/180)    # [-6,154]
        q[35] = max(min(q[35], 154*np.pi/180), -6*np.pi/180)    # [-6,154]
        
        
        
        skeleton.setPositions(q)
    gui.nativeAPI().renderSkeleton(skeleton)
    motion_refined[:,i] =  skeleton.getPositions()
    print(j)
    
    
    # time.sleep(0.02)


# Smooth the poses
for i in range(motion_refined.shape[0]):
    motion_refined[i,:] = savgol_filter(motion_refined[i,:], 21, 5)


In [ ]:
# 2D Constrain and not constrain: see the difference in gradient descend
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.plot(motion_refined[42,:], motion_refined[42,:], motion_refined[42,:], c='r', marker='.')
ax.scatter(motion.poses[42,:], motion.poses[42,:], motion.poses[42,:], c='b', marker='.')

# Set labels and title
ax.set_xlabel('shoulder_r_x')
ax.set_ylabel('shoulder_r_y')
ax.set_zlabel('shoulder_r_z')
ax.set_title('3D Surface Plot')

# Show plot
plt.show()

In [ ]:
# 3D Constrain and not constrain: see the difference in gradient descend
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.scatter(motion_refined[42,:], motion_refined[43,:], motion_refined[44,:], c='r', marker='.')
ax.scatter(motion.poses[42,:], motion.poses[43,:], motion.poses[44,:], c='b', marker='.')

# Set labels and title
ax.set_xlabel('shoulder_r_x')
ax.set_ylabel('shoulder_r_y')
ax.set_zlabel('shoulder_r_z')
ax.set_title('3D Surface Plot')

# Show plot
plt.show()

In [ ]:
# Replay motions from .csv
import csv
import time
import numpy as np
sub = "103"
skeleton = nimble.biomechanics.OpenSimParser.parseOsim('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/bioamass_v1.0/CMU/'+sub+'/ab_fits/Models/optimized_scale_and_markers.osim',\
                                                                '/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/skel_models_v1.1/Geometry/').skeleton

# CMU_01_11_ik -> very bad

motion_csv = np.loadtxt("data/dataset49/CMU_103_03_ik.csv",delimiter=",")

for i in range(motion_csv.shape[0]):
    skeleton.setPositions(motion_csv[i,:])
    gui.nativeAPI().renderSkeleton(skeleton)
    time.sleep(0.04)

In [ ]:
# Plot 2d positions
import matplotlib.pyplot as plt
motion_d49  = np.loadtxt("data/dataset49/CMU_114_13_ik.csv",delimiter=",")
motion_d49p = np.loadtxt("data/dataset49+/CMU_114_13_ik.csv",delimiter=",")

angles = []
for i in range(skeleton.getNumJoints()):
    j = skeleton.getJoint(i)
    for k in range(j.getNumDofs()):
        angles.append(j.getName()+'_'+j.getDofName(k))

i = 16

fig, axs = plt.subplots(1,1)
fig.suptitle(angles[i])

axs.plot(motion_d49[:,i]*180/np.pi)
axs.plot(motion_d49p[:,i]*180/np.pi)
# axs[0, 1].set_title('Cosine')
# axs[0, 1].set_ylim([-1.1, 1.1])

plt.tight_layout()
plt.savefig('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/tmp/'+angles[i]+'.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter

t = motion.timestamps

angles = []
for i in range(skeleton.getNumJoints()):
    j = skeleton.getJoint(i)
    for k in range(j.getNumDofs()):
        angles.append(j.getName()+'_'+j.getDofName(k))
    
i = 2

# p = motion.poses[i,:].copy()
p = motion_refined[i,:].copy()
p_norm = (motion.poses[i,:]%(2*np.pi))/np.pi

# Remove switching
for k in range(1,len(t)): 
    if p[k] - p[k-1] > 2*np.pi - 0.2:
        p[k] -= 2*np.pi
    elif p[k] - p[k-1] < -2*np.pi + 0.2:
        p[k] += 2*np.pi
    
win = 21
p = savgol_filter(p, win, 5)
d = np.diff(p)
dt = np.diff(t)
v = d/dt
v = savgol_filter(v, win, 5)
a = np.diff(v)/dt[1:]
a = savgol_filter(a, win, 5)

cp = np.cos(p)
sp = np.sin(p)

fig, axs = plt.subplots(2, 3)
fig.suptitle(angles[i])

axs[0, 0].plot(t, p/np.pi)
axs[0, 0].set_title('Angular position (pi)')

axs[0, 1].plot(t, cp)
axs[0, 1].set_title('Cosine')
axs[0, 1].set_ylim([-1.1, 1.1])


axs[0, 2].plot(t, cp)
axs[0, 2].set_title('Sine')
axs[0, 2].set_ylim([-1.1, 1.1])

axs[1, 0].plot(t[1:], d)
axs[1, 0].set_title('Difference')

axs[1, 1].plot(t[1:], v/np.pi)
axs[1, 1].set_title('Angular Velocity (pi)')

axs[1, 2].plot(t[2:], a/np.pi)
axs[1, 2].set_title('Angular Acceleration (pi)')

# axs[1, 0].plot(t, p_norm)
# axs[1, 0].set_title('Angular position normalized')

plt.tight_layout()
plt.savefig('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/tmp/'+angles[i]+'.png')
plt.show()

In [39]:
# Read and stack the entire dataset
import os
import numpy as np

dataset = '/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/dataset49+_a/'
d = None
for s in os.listdir(dataset):
    if d is None:
        d = np.loadtxt(dataset+s,delimiter=',')
    else:
        d = np.vstack([d,np.loadtxt(dataset+s,delimiter=',')])
np.savetxt('tmp/dataset49+_a.csv',d,delimiter=',')
d.shape

(740283, 49)

In [37]:
# Save acceleration and velocity of each file
import os
import numpy as np
import matplotlib.pyplot as plt

dataset = '/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/dataset49+/'
for s in os.listdir(dataset):
    file = np.loadtxt(dataset+s,delimiter=',')

    from scipy.signal import savgol_filter

    t = np.loadtxt(dataset.replace('dataset49+','bioamass_timestamps')+s,delimiter=',')
    V = np.zeros((len(t)-1,49))
    A = np.zeros((len(t)-2,49))
    for i in range(49):
        # p = motion.poses[i,:].copy()
        p = file[:,i].copy()
        
        win = 101
        if len(t) < 101:
            win = int(len(t)/3)
        
        p = savgol_filter(p, win, 5)
        d = np.diff(p)
        dt = np.diff(t)
        v = d/dt
        v = savgol_filter(v, win, 5)
        a = np.diff(v)/dt[1:]
        a = savgol_filter(a, win, 5)

        V[:,i] = v
        A[:,i] = a

        # cp = np.cos(p)
        # sp = np.sin(p)
        
        # fig, axs = plt.subplots(3,1)
        # fig.suptitle(str(i))
        # axs[0].plot(t,p)
        # axs[0].set_title('p')
        # axs[0].set_ylim([-3.14, 3.14])
        # axs[1].plot(t[1:],v)
        # axs[1].set_title('v')
        # axs[2].plot(t[2:],a)
        # axs[2].set_title('a')

    # d = np.vstack([d,np.loadtxt(dataset+s,delimiter=',')])
    np.savetxt(dataset.replace("49+","49+_v")+s,V,delimiter=',')
    np.savetxt(dataset.replace("49+","49+_a")+s,A,delimiter=',')
d.shape


(5974,)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.scatter(d[:,32], d[:,33], d[:,34], c='r', marker='.')

# Set labels and title
ax.set_xlabel('shoulder_r_x')
ax.set_ylabel('shoulder_r_y')
ax.set_zlabel('shoulder_r_z')
ax.set_title('3D Surface Plot')

# Show plot
plt.show()

In [ ]:
# Compute the velocity
import os
import numpy as np

dataset = '/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/dataset49/'
d = None
for s in os.listdir(dataset):
    if d is None:
        d = np.loadtxt(dataset+s,delimiter=',')
    else:
        d = np.vstack([d,np.loadtxt(dataset+s,delimiter=',')])
d.shape